# 判別分析

## ライブラリなど読み込み

In [ ]:
import numpy as np               
import matplotlib.pyplot as plt
from common import mlbench as ml                      #  講習会のためのスクリプト
from common.contourFittedClass import cplot    #  等高線プロットのためのスクリプト

## サポートベクトルマシン(SVM)

## データ生成     

In [ ]:
X,y = ml.twoDnormals(200, cl=2, sd=1)  # トレーニングデータ

In [ ]:
plt.scatter(X[y==0,0],X[y==0,1], c='red')
plt.scatter(X[y==1,0],X[y==1,1], c='blue')
plt.show()

## サポートベクトルマシン
 sklearn.svm.SVC  

In [ ]:
from sklearn.svm import SVC                 #  モジュール読み込み

In [ ]:
sv = SVC(kernel="linear", coef0=1)      #  線形判別モデル
sv.fit(X,y)                                     # サポートベクトルマシンでデータを学習

In [ ]:
py = sv.predict(X)    # ラベル予測
np.mean(py != y)     # トレーニング誤差

In [ ]:
1-sv.score(X,y)         # トレーニング誤差： score を使うこともできる

In [ ]:
# 設定パラメータの詳細
sv.get_params()

## 予測ラベル・テスト誤差

In [ ]:
tX,ty = ml.twoDnormals(1000, cl=2, sd=1)   # テストデータ  
py = sv.predict(tX)     # 予測ラベル
np.mean(py != ty)       # テスト誤差

In [ ]:
1-sv.score(tX,ty)        # テスト誤差

## プロット

In [ ]:
cplot(sv,X,y)

## カーネルSVM
- sklearn.svm.SVC
- kernel オプション

## データ

In [ ]:
X,y = ml.spirals(300, cycles=1,sd=0.15)         # トレーニングデータ
tX,ty = ml.spirals(1000,cycles=1,sd=0.15)    # テストデータ

In [ ]:
# トレーニングデータのプロット
plt.scatter(X[y==0,0],X[y==0,1], c='red')
plt.scatter(X[y==1,0],X[y==1,1], c='blue')
plt.show()

## SVM：カーネルは2次多項式

In [ ]:
sv2 = SVC(kernel="poly",degree=2, gamma=1, coef0=1).fit(X,y)  # 設定と学習

In [ ]:
# プロット：2次多項式カーネル
print("テスト誤差：", 1-sv2.score(tX,ty))
cplot(sv2,X,y)

## SVM：カーネルは3次多項式

In [ ]:
sv3 = SVC(kernel="poly",degree=3,gamma=1,coef0=1).fit(X,y)   # 設定と学習

In [ ]:
# プロット：3次多項式カーネル
print("テスト誤差：", 1-sv3.score(tX,ty))
cplot(sv3,X,y)

## モデルパラメータの選択

## 交差検証法
sklearn.model_selection.cross_validate

In [ ]:
# モジュール読み込み
from sklearn.model_selection import cross_validate 

## データ

In [ ]:
X,y    = ml.spirals(200,  cycles=1.2, sd=0.1)       # トレーニングデータ
tX,ty = ml.spirals(1000,cycles=1.2, sd=0.1)     # テストデータ 

In [ ]:
plt.scatter(X[y==0,0],X[y==0,1], c='red')
plt.scatter(X[y==1,0],X[y==1,1], c='blue')
plt.show()

## 交差検証法

In [ ]:
sv = SVC(kernel="rbf", gamma=10, C=1)           # ガウスカーネルのSVMモデルを設定

In [ ]:
cv = cross_validate(sv,X,y,scoring='accuracy',cv=5)   # 交差検証法：K=5
1-cv['test_score']                                                             # 交差検証法の各ブロックごとの誤差

In [ ]:
1-np.mean(cv['test_score'])        # テスト誤差の推定値

## ガウスカーネル：gamma の候補

In [ ]:
# ガウスカーネルのgammaの候補： データの間隔から候補を決める
from scipy.spatial import distance
dm = distance.pdist(X)                     # 距離行列の計算

# 候補の値： データ間の距離のパーセント点
cg = 1/np.percentile(dm,np.arange(1,100,2))**2 
cg

## 交差検証法：gamma の決定

In [ ]:
cvg = np.array([])
for g in cg:                      # 各gammaごとに検証誤差を計算
    sv = SVC(kernel="rbf", gamma=g, C=1)
    cv = cross_validate(sv,X,y,scoring='accuracy',cv=5)
    cvg = np.r_[cvg, np.mean(cv['test_score'])]

In [ ]:
# 最適な  gamma の選択
cverr = 1-cvg                                         # 検証誤差
opt_gamma = cg[np.argmin(cverr)]  # 最適なgamma
opt_gamma

## 最適な gamma 

In [ ]:
sv = SVC(kernel="rbf",C=1,gamma=opt_gamma)
sv.fit(X,y)           #  学習
print('テスト誤差: ', 1-sv.score(tX,ty))
cplot(sv,X,y)

## 小さい gamma 

In [ ]:
sv = SVC(kernel="rbf",C=1,gamma=cg.min())
sv.fit(X,y)                     # 学習
print("テスト誤差: ", 1-sv.score(tX,ty))            
cplot(sv,X,y)

## 大きい gamma

In [ ]:
sv = SVC(kernel="rbf",C=1,gamma=cg.max())
sv.fit(X,y)             # 学習
print('テスト誤差: ',1-sv.score(tX,ty))          
cplot(sv,X,y)